In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa
import re
import findspark
findspark.init('/home/dave/spark-2.4.1-bin-hadoop2.7/')
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql import *
import sys
import os
from IPython.display import display, HTML
from pyspark.sql.functions import pandas_udf, PandasUDFType
%load ./py/loc_av.py


datapath = "/home/dave/Documents/CS691/proj/data/snotel/"
s1049="SNOTEL_1049_FORESTDALE_CREEK"    
s539="SNOTEL_539_INDEPENDENCE_CAMP"
s1050="SNOTEL_1050_HORSE_MEADOW"        
s540="SNOTEL_540_INDEPENDENCE_CREEK"
s1051="SNOTEL_1051_BURNSIDE_LAKE"       
s541="SNOTEL_541_INDEPENDENCE_LAKE"
s1067="SNOTEL_1067_CARSON_PASS"         
s615="SNOTEL_615_MARLETTE_LAKE"
s340="SNOTEL_340_BIG_MEADOW"           
s652="SNOTEL_652_MT_ROSE"
s356="SNOTEL_356_BLUE_LAKES"          
s428="SNOTEL_428_CSS_LAB"              
s724="SNOTEL_724_RUBICON"
s809="SNOTEL_809_TAHOE_CITY"
s463="SNOTEL_463_ECHO_PEAK"            
s834="SNOTEL_834_TRUCKEE"
s508="SNOTEL_508_HAGANS_MEADOW"        
s848="SNOTEL_848_WARD_CREEK"
s518="SNOTEL_518_HEAVENLY_VALLEY"

file1 = datapath+s848
file2 = datapath+s518


truckee = [s541,s540,s428,s834]
west_shore = [s809,s848,s724]
south_shore = [s518,s508,s1050,s463]
mt_rose = [s340,s652,s615]
hope_valley = [s1051,s1049,s1067,s356]

print(file1)
print(file2)

spark = SparkSession.builder.appName("proj_exp").getOrCreate()

df = spark.read.text(file1)
df.show()